<a href="https://colab.research.google.com/github/danghoangminh/CS336.L12.KHCL/blob/master/Vector_Space/Vector_space_invert_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import date, timedelta

today_ = date.today()

# dd/mm/YY
today = str(today_.day) + "/" + str(today_.month) + "/" + str(today_.year)
today = datetime.datetime.strptime(today, '%d/%m/%Y')

data_dir = "/content/drive/My Drive/Colab Notebooks/CS336/Vector_Space/data/"
last_month = date(today_.year, today_.month - 2, today_.day)
delta = today_ - last_month
index = 0

for i in range(delta.days + 1):
    day = last_month + timedelta(days=i)
    day = str(day.day) + "-" + str(day.month) + "-" + str(day.year)
    url = "https://tuoitre.vn/xem-theo-ngay/" + day + ".htm"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    titles = soup.findAll('h3', class_='title-news')
    links = [link.find('a').attrs["href"] for link in titles]
    for link in links:
        news = requests.get("https://tuoitre.vn" + link)
        soup = BeautifulSoup(news.content, "html.parser")
        try:
            title = soup.find("h1", class_="article-title").text
            abstract = soup.find("h2", class_="sapo").text
            body = soup.find("div", id="main-detail-body")
            paras = body.findChildren("p", recursive=False)
            content = ""
            for p in paras:
                content += p.text
            with open(data_dir + str(index) + '.txt', 'w', encoding='utf-8') as f:
                f.write(content)
                index += 1
        except:
            continue

In [4]:
import os
import re
import numpy as np

path = "/content/drive/My Drive/Colab Notebooks/CS336/Vector_Space/data/"

def sorter(item):
    id = list(item[1])
    return float(id[0])

def load_data_in_a_directory(path):
    list_str = []
    files = os.listdir(path)
    for file in files:
        f = open(path+file,"r",encoding="utf8")
        content_file = f.read()
        content_file = re.sub("\,|\.|\-|\'|\"|\_|\+|\?|\!|\@|\#|\$|\%|\;|\[|\]|\/|\:|\)|\(","",content_file)
        content_file = content_file.lower()
        content_file = content_file.split()
        list_str.append(content_file) 
    return (list_str,files)

def build_dictionary(contents):
    dictionary = set()
    for content in contents:
        dictionary.update(content)
    return dictionary

def tf_weighting(vocab, contents,query_content):
    k={}
    TF = np.zeros(((len(vocab)),len(contents)))
    for i, word in enumerate(vocab):
        if word in query_content:
            for j, content in enumerate(contents):
                TF[i,j] = content.count(word)
                if k.get(word,None) is None:
                    k[word] = {(i,j)}
                else:
                    k[word].add((i,j))
        else:
            for j, content in enumerate(contents):
                TF[i,j] = content.count(word)
    TF = TF / np.sum(TF, axis =0)
    return TF,k

arr1 = []
query = "trump biden"
query_content = query.split()
contents, paths = load_data_in_a_directory(path)
vocab = build_dictionary(contents)
TF,k = tf_weighting(vocab, contents,query_content)
DF = np.sum(TF!=0, axis=1)
IDF = 1+ np.log2(len(contents)/DF)
IDF = np.array([IDF]).T 
TF_IDF = TF * IDF

for i in range(len(query_content)):
    arr= k.get(query_content[i])
    arr = (list(arr))
    arr1.append(arr)
inverted_index = {}
posting = []
for i, querys in enumerate(query_content):
    m = arr1[i]
    for j in m:
        x = j[0]
        y = j[1]
        l = TF_IDF[x,y]
        if inverted_index.get(y, None) is None:
            inverted_index[y] = {l}
        else:
            g = list(inverted_index.get(y))
            f = float(l)+ float(g[0])
            inverted_index.pop(y)
            inverted_index[y] = {f}
arr2 = list(inverted_index.items())
arr2 = np.array(arr2)
rank = sorted(arr2,key=sorter)
topk = 3
for i in range(1,topk):
    print('Van ban gan thu', i, 'la: ', ' '.join(contents[rank[-1*i][0]]))

Van ban gan thu 1 la:  cuộc khẩu chiến giữa ông trump và ông biden đang ngày càng tăng cấp khi thời gian dần trôi về mốc 299 màn tranh luận trực tiếp đầu tiên giữa hai người có ý nghĩa quan trọng quyết định ai sẽ là người giành được cảm tình của các cử tri đang còn phân vânvấn đề sức khỏe của ông biden cũng sẽ được hé lộ thông qua các cuộc tranh luận với đối thủ trump vốn nhỏ hơn 4 tuổi theo hãng thông tấn afptôi sẽ mạnh mẽ yêu cầu làm xét nghiệm chất kích thích với biden buồn ngủ trước hoặc sau cuộc tranh luận tuần tới tất nhiên tôi cũng sẽ gật đầu làm xét nghiệm ông trump thách thức đối thủ trên twitter cá nhân ngày 279theo tổng thống mỹ các màn tranh luận của ông biden thường có tiếng là không được trơn tru cho lắm ông cho rằng chỉ có chơi thuốc mới giúp cựu phó tổng thống biden tạo ra khác biệt nhưng không đưa ra được bằng chứng gìbản thân ông biden trước đó đã hâm nóng cuộc khẩu chiến khi so sánh ông trump với bộ trưởng truyền thông của phát xít đức “ông ta giống như goebbels khi 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
